In [1]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import pickle


Bad key "text.kerning_factor" on line 4 in
C:\Users\Karan\Anaconda3\envs\forecast\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# load the true value dictionary from file
gb_dict = pickle.load( open( "store_class_dictionary_updated.pickle", "rb" ) )

In [12]:
list_del = []
list_k = []
list_r = []
count = 0
for key,val in gb_dict.items():
        sales_df = val.values
        index_df = val.index
        df = pd.DataFrame(data = sales_df, index= index_df, columns = ['y'])
        #year_
        #count_year = df[df['y']==0].count()
        year_df = val.loc[val.index<'2017'] 
        count_in_group = year_df[year_df['totalSales'] == 0]['totalSales'].count()
        
#         if count_in_group < 52*5*.3:
#             list_del.append(key)
        # thresholding on 50%: 1052
        # thresholding on 30%: 969
        
        if count_in_group >= 250:
            list_del.append(key)
        
#         df.drop(indexNames , inplace=True)
#         tmp2 = df.groupby(df.index.year).agg(['count'])
#         if 2015 in tmp2.index:
#             k = tmp2.loc[2015].values
#             list_k.append(k)
#         if 2016 in tmp2.index:
#             r = tmp2.loc[2016].values
#             list_r.append(r)
#         if (k+r) < 30:
#             list_del.append(key)  
#         Removed 889
        

In [13]:
print(len(list_del))
# for k, v in gb_dict.items():
    

667


In [16]:
zero=1.5
for key,val in gb_dict.items():
        sales_df = val.values
        index_df = val.index
        df = pd.DataFrame(data = sales_df, index= index_df, columns = ['y'])
        #indexNames = df[ df['y'] == 0 ].index
        df = df.replace(to_replace = 0, value = zero) 
        neg_df = df.loc[df['y'] < 0] 
        index_drop = neg_df.index
        final_df = df.drop(index_drop)
        final_df['y'] = np.log(final_df['y'])
        gb_dict[key] = final_df['y']
        

In [14]:
for key in list_del:
    del gb_dict[key]

In [17]:
#saving the model
with open("gb_dict_log_imputed.pickle", "wb") as f:
    pickle.dump(gb_dict, f)

In [15]:
len(gb_dict)

1550

In [ ]:
len(valid_sales)
    
    

In [ ]:
#Impute missing values further
#creating dictionaries to store them as pickle files after training the model
pred_sales = {}
valid_sales= {}
fcst_model = {}
model_all2 = {}
#training the Prophet model over each store-class combination
for key,val in gb_dict.items():
    sales_df = val.values
    index_df = val.index
    df = pd.DataFrame(data = sales_df, index= index_df, columns = ['y'])
    # splitting data into training and validation in 75:25 ratio
    train = df[0:260]
    test = df[260:] 
    model = Prophet()
    model.fit(train.reset_index().rename(columns={'index':'ds'}))
    fcst = model.predict(df = test.reset_index().rename(columns={'index':'ds'}))
    fcst_model[key] = fcst
    y_pred = fcst['yhat']
    y_test = test['y']
    pred_sales[key] = y_pred
    valid_sales[key] = y_test
    model_all2[key] = model

In [ ]:
#saving the model
with open("store_class_imputed_log_noneg_model.pickle", "wb") as f:
    pickle.dump(model_all2, f)

In [ ]:
#saving the true values of validation set
with open("store_class_imputed_log_noneg_true.pickle", "wb") as f:
    pickle.dump(valid_sales, f)

In [ ]:
#saving the predicted values of validation set
with open("store_class_dictionary_imputed_log_noneg_fcst_model.pickle", "wb") as f:
    pickle.dump(fcst_model, f)

In [ ]:
#saving the predicted values of validation set
with open("store_class_dictionary_imputed_log_noneg_pred_sales.pickle", "wb") as f:
    pickle.dump(pred_sales, f)

In [ ]:
# load the model dictionary from file
md_pred2 = pickle.load( open( "store_class_imputed_log_noneg_model.pickle", "rb" ) )

In [ ]:
# load the true value dictionary from file
true_pred = pickle.load( open( "store_class_imputed_log_noneg_true.pickle", "rb" ) )

In [ ]:
# load the predicted value dictionary from file
fcst_pred = pickle.load( open( "store_class_dictionary_imputed_log_noneg_fcst_model.pickle", "rb" ) )

In [ ]:
# load the predicted value dictionary from file
pred_sal = pickle.load( open( "store_class_dictionary_imputed_log_noneg_pred_sales.pickle", "rb" ) )

In [ ]:
#creating lists for plotting
lt4 = list(true_pred.values())
test1 = lt4[0]

In [ ]:
s = list(map(list,true_pred.values()))
j = list(map(list, pred_sal.values()))

In [ ]:
print((len(s[0])))

In [ ]:
print(s[0])

In [ ]:
for i in range(0,len(s)):
    z.append(np.exp(s[i]))

In [ ]:
w = []
for i in range(0,len(j)):
    w.append(np.exp(j[i]))

In [ ]:
w[0]

In [ ]:
mse

In [ ]:
mae

In [ ]:
#calculating mse and mae
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = []
mae = []
for i in range(0,len(w)):
    mse.append(mean_squared_error(y_true=z[i],y_pred=w[i]))
    mae.append(mean_absolute_error(y_true=z[i],y_pred=w[i]))

In [ ]:
#calculating mape
mape = []
zero=1e-24
def mean_absolute_percentage_error2(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true+zero))) * 100
for i in range(0,len(w)):
    mape.append(mean_absolute_percentage_error2(y_true=z[i],y_pred=w[i]))

In [ ]:
#saving evaluation metrics to csv via dataframe
df2 = pd.DataFrame(list(zip(mse, mae,mape)), columns =['MSE', 'MAE', 'MAPE']) 
df2.to_csv('evaluation_exp.csv')

In [ ]:
s[1]

In [ ]:
j[1]

In [ ]:
lt3 = list(pred_sal.values())

In [ ]:
#Using the summation matrix to get predictions of every combination in the hierarchy
summation = pickle.load(open("agg_combo.pickle", "rb"))

In [ ]:
#initializing one leaf node
b = lt3[0]
b = b.to_numpy()
b = b.reshape(-1,53)

In [ ]:
#defining other leaf nodes
for i in range(1,len(lt3)):
    numpy_b = lt3[i].to_numpy()
    numpy_s = numpy_b.reshape(-1,53)
    b = np.append(b, numpy_s, axis = 0)

In [ ]:
#converting to numpy for enabling multiplication
S_arr = summation.to_numpy()

In [ ]:
#getting the predictions
ypred = np.dot(S_arr,b)

#does not give good result